In [1]:


def get_json(response):
    # Define the regular expression for categories from the main page
    pattern = re.compile(r'window\.yelp\.react_root_props\s*=\s*(\{.*?\});')

    # Search for the pattern in the input string
    match = pattern.search(response.text)

    # Extract the JSON string
    if match:
        json_str = match.group(1)

        # Load the JSON data
        j = json.loads(json_str)

        # Print or process the JSON data
    else:
        print("No match found")
    return j


def write_json(data, filename):
    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Data successfully written to {filename}")
    except Exception as e:
        print(f"An error occurred: {e}")
        
        
def find_key_path(json_obj, target_key, current_path=""):
    """
    Finds the path to the target key in a nested JSON object.

    Parameters:
    json_obj (dict or list): The JSON object to search.
    target_key (str): The key to find.
    current_path (str): The current path being explored (used in recursion).

    Returns:
    list: A list of paths where the target key is found.
    """
    paths = []

    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            new_path = f"{current_path}/{key}" if current_path else key
            if key == target_key:
                paths.append(new_path)
            paths.extend(find_key_path(value, target_key, new_path))
    elif isinstance(json_obj, list):
        for index, item in enumerate(json_obj):
            new_path = f"{current_path}[{index}]"
            paths.extend(find_key_path(item, target_key, new_path))

    return paths

In [2]:
import pandas as pd
import re
import requests
import json
from itertools import cycle
from bs4 import BeautifulSoup

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}
home_page = "https://www.yelp.com/"
response = requests.get(home_page, headers=headers)

home_page_json = get_json(response)


In [ ]:
# write_json(home_page_json, 'home_page_json.json')

In [3]:


initial_cat = home_page_json["legacyProps"]["category_tiles_props"]["initialCategories"]

all_cat = []
for i in initial_cat:
    all_cat.append(home_page+i['category']['url'])
    

more_cat = home_page_json["legacyProps"]["category_tiles_props"]["moreCategories"]
for i in more_cat:
    all_cat.append(home_page+i['category']['url'])

In [4]:
dic = {
    'name':[],
    'address':[],
    'tele_num':[],
    'link':[],
    'mainweb_link':[],
    'pagination_link':[],
    'category':[],
    'latitude':[],
    'longitude':[],
    'hours':[],
    'socials':[],
    'img_url':[]
}

dic_1 = {
}

In [6]:

for cat_url in all_cat[:20]:
    ct = 0
    for num_page in range(10):
        category_url = cat_url+ f'&start={ct}'
        pagination_link = category_url
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
                }
        response = requests.get(category_url, headers=headers)
        cat_page_json = get_json(response)
        
        loc = cat_page_json["legacyProps"]["searchAppProps"]["searchPageProps"]["rightRailProps"]["searchMapProps"]["mapState"]["markers"]
        for i in loc:
            if 'resourceType' in i and i["resourceType"] == 'business':
                title_key = i["title"]
                latitude = i["location"]["latitude"]
                longitude = i["location"]["longitude"]

                print(i["title"])
                print(latitude)
                print(longitude)

                if title_key not in dic_1:
                    dic_1[title_key] = []

                dic_1[title_key].extend([latitude, longitude])
        
        
        
        
        sub_links = cat_page_json["legacyProps"]["searchAppProps"]["searchPageProps"]["mainContentComponentsListProps"]
        for i in sub_links:
            if "searchResultBusiness" in i and i["searchResultBusiness"]["isAd"] == False:
                per_hotellinks = 'https://www.yelp.com/'+ i["searchResultBusiness"]["businessUrl"]
                print(per_hotellinks)
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                    'Accept-Language': 'en-US,en;q=0.9',
                    'Accept-Encoding': 'gzip, deflate, br',
                    'Connection': 'keep-alive',
                    'Upgrade-Insecure-Requests': '1'
                        }
                response = requests.get(per_hotellinks, headers=headers)
                per_hotellinks_json = get_json(response)
                category_ = per_hotellinks.split('=')[-1]
                print(pagination_link)
                print(category_)
                
                name = per_hotellinks_json["legacyProps"]["bizDetailsProps"]["bizDetailsPageProps"]["businessName"]
#                 write_json(per_hotellinks_json, f'per_hotellinks_json{cout}.json')
                print(name)
                soup = BeautifulSoup(response.content,'html.parser')
                add_match = ''
                if soup.find('p',class_='y-css-dg8xxd'):
                    add_match = soup.find('p',class_='y-css-dg8xxd').text
                    print(add_match)
                else:
                    print("No address found.")
                    
                    
                

                link_pattern = r'rel="noopener.*?">(.*?)</a></p>'
                # Search for the pattern in the HTML content
                link_match = re.search(link_pattern, response.text)

                # If a match is found, print the link text
                web_link = ''
                if link_match:
                    print(link_match.group(1))
                    web_link = link_match.group(1)
                else:
                    print("No link text found.")
                    
                
                
                tele_pattern = r'Phone number</p>.*?(\(.*?)</p>'
                # Search for the pattern in the HTML content
                tele_match = re.search(tele_pattern,response.text)

                # If a match is found, print the phone number
                web_num = ''
                if tele_match:
                    print(tele_match.group(1))
                    web_num = tele_match.group(1)
                else:
                    print("No phone number found.")
                    
                dic['name'].append(name)
                dic['address'].append(add_match)
                dic['tele_num'].append(web_num)
                dic['link'].append(web_link)
                dic['mainweb_link'].append(per_hotellinks)
                dic['pagination_link'].append(pagination_link)
                dic['category'].append(category_)
                dic['hours'].append('')
                dic['socials'].append('')
                dic['img_url'].append('')
        ct+=10
        

for n in dic['name']:
    long, lat = dic_1.get(n, ('', ''))
    dic['latitude'].append(lat)
    dic['longitude'].append(long)       
        
        


Bueokae Korean Restaurant - Love's Kitchen
43.89038
-79.44035
Chillax & Co
43.86121099118103
-79.43554637388071
The Big Casserole
43.88079293749161
-79.3941804854628
Glass Kitchen
43.8426811
-79.3869911
Jatujak
43.87764840184963
-79.43814855767062
Kyushu Yakitori Izakaya
43.80369187431343
-79.420159
Isaan Der - Thai Kitchen & Bar
43.8483064
-79.38221159999999
Hongdae Korean Restaurant
43.85092059999999
-79.433599
Hong Kong Wing Tai Restaurant
43.8824479
-79.43891767055204
BanBan
43.8460666
-79.37947779999999
https://www.yelp.com//biz/bueokae-korean-restaurant-loves-kitchen-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=0
Restaurants
Bueokae Korean Restaurant - Love's Kitchen
10737 Yonge St Unit 2/3 Richmond Hill, ON L4C 3E3 Canada
No link text found.
(905) 237-0737
https://www.yelp.com//biz/chillax-and-co-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=0
Restaura

https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=20
Restaurants
Jin Korean Noodles
230 Commerce Valley Drive E Unit 7 Markham, ON L3T 7Y3 Canada
jknoodles.com
(905) 764-5922
https://www.yelp.com//biz/bap-zip-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=20
Restaurants
Bap Zip
1285 Elgin Mills Road E Richmond Hill, ON L4S 0B5 Canada
No link text found.
(905) 737-2288
https://www.yelp.com//biz/mikaku-udon-bar-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=20
Restaurants
Mikaku Udon Bar
360 Highway 7 E Unit 10 Richmond Hill, ON L4B 3Y7 Canada
No link text found.
(905) 707-8787
https://www.yelp.com//biz/loon-fong-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=20
Restaurants
Loon Fong
330 Highway 7 Suite 106 Richmond Hill, ON L4B 3P8 Canada
No link text found.
(905) 763-888

https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=50
Restaurants
Mi Ne Japanese Restaurant
10815 Yonge Street Richmond Hill, ON L4C 3E3 Canada
minesushi.ca
(905) 883-6616
https://www.yelp.com//biz/stroganoff-restaurant-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=50
Restaurants
Stroganoff Restaurant
10 Headdon Gate Unit 2-3 Richmond Hill, ON L4C 9W9 Canada
stroganoff.co
(905) 780-7575
https://www.yelp.com//biz/l%C3%A1-sen-vietnamese-cuisine-toronto?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=50
Restaurants
Lá Sen Vietnamese Cuisine
155 Yorkland Boulevard Toronto, ON M2J Canada
lasenvietcuisine.com
(416) 502-2488
https://www.yelp.com//biz/lee-brothers-bbq-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=50
Restaurants
Lee Brothers BBQ
550 Highway 7 Unit 22 Floor 2 Richmond Hill, ON 

10909 Yonge Street Richmond Hill, ON L4C 3E3 Canada
phongason.com
(905) 237-8889
https://www.yelp.com//biz/chillax-coffee-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=70
Restaurants
Chillax Coffee
10650 Leslie Street Unit 2 Richmond Hill, ON L4S 0B9 Canada
chillaxcoffee.ca
(905) 770-6668
https://www.yelp.com//biz/3-mariachis-express-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=70
Restaurants
3 Mariachis Express
16 Vogell Road Unit L Richmond Hill, ON L4B 3K4 Canada
3mariachis.ca
(905) 237-6006
https://www.yelp.com//biz/hapinoy-ng-bayan-richmond-hill?osq=Restaurants
https://www.yelp.com//search?find_desc=Restaurants&find_loc=Richmond+Hill&start=70
Restaurants
HaPinoy Ng Bayan
437 centre street East Richmond Hill, ON L4C 1B8 Canada
hapinoyngbayan.ca
(289) 748-5094
Treasure Garden
43.91356584348192
-79.44810559999999
Daimaru Sukiyaki
43.8152702
-79.295929899999

https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=0
Shopping
Vaughan Mills
1 Bass Pro Mills Drive Vaughan, ON L4K 5W4 Canada
vaughanmills.com
(905) 879-2110
https://www.yelp.com//biz/pacific-mall-markham-2?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=0
Shopping
Pacific Mall
4300 Steeles Avenue E Markham, ON L3R 0Y5 Canada
pacificmalltoronto.ca
(905) 470-8785
https://www.yelp.com//biz/chic-thrills-stouffville?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=0
Shopping
Chic Thrills
6316 Main Street Stouffville, ON L4A 1G8 Canada
chicthrills.ca
(905) 642-3339
https://www.yelp.com//biz/times-square-mall-richmond-hill?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=0
Shopping
Times Square Mall
550 Highway 7 E Richmond Hill, ON L4B Canada
timessquarerichmondhill.ca
(905) 882-4360
https://www.yelp.com//biz/cf-fairview-mall-north-york?osq=Sh

https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=30
Shopping
Marshall HomeSense
9350 Yonge Street Richmond Hill, ON L4C 5G2 Canada
marshalls.ca/en
(905) 884-8057
https://www.yelp.com//biz/downsview-park-merchants-market-and-farmers-market-north-york?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=30
Shopping
Downsview Park Merchants Market & Farmers Market
40 Carl Hall Road North York, ON M3K 2C1 Canada
dpmarket.com
(416) 638-2525
https://www.yelp.com//biz/muji-markham?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=30
Shopping
Muji
5000 Highway 7 E Markham, ON L3R 4M9 Canada
muji.com
No phone number found.
https://www.yelp.com//biz/haute-classics-woodbridge?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=30
Shopping
Haute Classics
8611 Weston Road Unit 28 Woodbridge, ON L4L 9P1 Canada
hauteclassics.com
(416) 922-7900
https://www.yelp

https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=50
Shopping
Tokyo Papa
3255 Highway 7 East Unit 205 Markham, ON L3R 3P9 Canada
tokyopapa.com
(905) 604-3918
Joanna's Bridal & Fashion
43.809316043522806
-79.26904071596854
Aritzia
43.85485449999999
-79.436539
Second Chance Clothing and More
43.8349869
-79.4954558
Plato's Closet
43.737059
-79.250885
The Salvation Army Thrift Store
43.9028231
-79.4452839
Around The Block
43.75938685322875
-79.35864557392404
Dollarama
43.90038770802401
-79.394483
Shoppes of The Parkway
43.8460696
-79.3794793
Fast Time Watch & Jewellery Repair
43.8814934
-79.42992020000001
Shoppers Drug Mart
43.8633131775877
-79.4345066633613
https://www.yelp.com//biz/joannas-bridal-and-fashion-toronto?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=60
Shopping
Joanna's Bridal & Fashion
1571 Sandhurst Circle Unit 112 Toronto, ON M1V 1V2 Canada
joannasbridal.com
(416) 299-7274
https://www.yelp.com//biz/arit

Verified by Business 
torontohabitat.ca/restore/loca…
(416) 755-7353
https://www.yelp.com//biz/bayview-village-shopping-centre-north-york?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=80
Shopping
Bayview Village Shopping Centre
2901 Bayview Avenue North York, ON M2K 1E6 Canada
bayviewvillageshops.com
(416) 226-0404
https://www.yelp.com//biz/chez-jordan-woodbridge?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=80
Shopping
Chez Jordan
4040 Steeles Avenue W Suite 30 Woodbridge, ON L4L 4Y5 Canada
chezjordanbridal.com
(905) 265-1887
https://www.yelp.com//biz/village-gate-richmond-hill?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&find_loc=Richmond+Hill&start=80
Shopping
Village Gate
9665 Avenue Bayview Richmond Hill, ON L4C 9V4 Canada
No link text found.
(905) 770-5039
https://www.yelp.com//biz/bed-bath-and-beyond-richmond-hill?osq=Shopping
https://www.yelp.com//search?find_desc=Shopping&fi

120 Interchange Way Vaughan, ON L4K 5C3 Canada
daveandbusters.com
(905) 760-7600
https://www.yelp.com//biz/mandala-cafe-and-lounge-richmond-hill?osq=Nightlife
https://www.yelp.com//search?find_desc=Nightlife&find_loc=Richmond+Hill&start=10
Nightlife
Mandala Cafe & Lounge
10670 Yonge Street Richmond Hill, ON L4C 0C7 Canada
mandalacafe.ca
(905) 237-7969
https://www.yelp.com//biz/douyin-bar-scarborough?osq=Nightlife
https://www.yelp.com//search?find_desc=Nightlife&find_loc=Richmond+Hill&start=10
Nightlife
DouYin Bar
2901 Kennedy Road Scarborough, ON M1V 1S8 Canada
No link text found.
(647) 285-9825
https://www.yelp.com//biz/the-marlowe-restaurant-and-bar-richmond-hill?osq=Nightlife
https://www.yelp.com//search?find_desc=Nightlife&find_loc=Richmond+Hill&start=10
Nightlife
The Marlowe Restaurant and Bar
155 York Boulevard Suite 1 Richmond Hill, ON L4B 3B4 Canada
themarlowe.ca
(905) 771-8645
https://www.yelp.com//biz/take-one-karaoke-and-lounge-markham?osq=Nightlife
https://www.yelp.com//sea

UnboundLocalError: cannot access local variable 'j' where it is not associated with a value

In [7]:
df = pd.DataFrame(dic)
df.to_csv('yelp_data_4cat_withlong.csv')

In [6]:
dic

{'name': ['Bottega',
  'Fog Harbor Fish House',
  '606',
  'Marufuku Ramen',
  'Daeho Kalbijjim & Beef Soup',
  'Four Chairs',
  'Wooden Spoon',
  'Beit Rima',
  'Blind Butcher',
  'Fable',
  'Nopa',
  'The Snug',
  'The Hungry Spot',
  'Sisterita',
  'Pearl',
  'Sotto Mare',
  'Kothai Republic',
  'Oraan Thai Eatery',
  'Santeria',
  'Kitchen Story',
  'San Francisco Centre',
  'Sakura',
  'Foggy Notion',
  'Local Take',
  'Mai Do - San Francisco',
  'The Love of Ganesha',
  'My Favorite',
  'San Francisco Mercantile',
  'Love On Haight',
  'Sanko Kitchen Essentials',
  'Rare Device',
  'Wink SF',
  'Love Street Vintage',
  'Park Life',
  "Cliff's Variety",
  'Pureland',
  'Sezane',
  'The Perish Trust',
  'Relic Vintage',
  'Anime Pop',
  'Emporium Arcade Bar',
  'Temple Nightclub',
  'The Valencia Room',
  'F8 Nightclub and Bar',
  'Underground SF',
  'The View Lounge',
  'Butter',
  "Bimbo's 365 Club",
  'Rooftop 25',
  'Club Fugazi',
  'Bootie Mashup: SF',
  "The Emperor's Drawing